<a href="https://colab.research.google.com/github/neverix/gpt2-optimize/blob/main/GPT2DreamModMod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the EasyTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook


In [ ]:
DEBUG_MODE = False
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "vscode"

In [ ]:
import os

if IN_COLAB:
    os.system("pip install git+https://github.com/neelnanda-io/Easy-Transformer.git")

In [ ]:
8# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm

import random
import time

import numpy as np
# from google.colab import drive
from pathlib import Path
import pickle
import os


import matplotlib.pyplot as plt

%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go

from torch.utils.data import DataLoader

from functools import *
import pandas as pd
import gc
import collections
import copy

# import comet_ml
import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets

In [ ]:
%%capture
# Install the right version of node - this is needed to install Svelte which is used to build components
# v16, an older version, seems to work more reliably than v18 on the systems I've tried it on.
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
# Get an up-to-date PySvelte, deleting old versions if present
!touch PySvelte
!rm -r PySvelte
!git clone https://github.com/neelnanda-io/PySvelte.git
import sys

sys.path.append("/content/PySvelte")

In [ ]:
import pysvelte

In [ ]:
!pip install git+https://github.com/redwoodresearch/Easy-Transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/redwoodresearch/Easy-Transformer to /tmp/pip-req-build-7cgfurpf
  Running command git clone --filter=blob:none --quiet https://github.com/redwoodresearch/Easy-Transformer /tmp/pip-req-build-7cgfurpf
  Resolved https://github.com/redwoodresearch/Easy-Transformer to commit 94ed3599b17209c69eb96973c8b61d8ee98a9dc9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00
  Created wheel for easy-transformer: filename=easy_transformer-0.1.0-py3-none-any.whl size=99240 sha256=8fe4819f3f4579b873f4b8f39f5119cbfcb8e2616b2701c4758fcf3a2ae674ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-b3n8e0ks/wheels/4c/94/e6/2d17fef9e311150cd3a162ead3d2571fb7e77d98392a6df733
Successfully built easy-transformer


In [ ]:
from easy_transformer.utils import (
    gelu_new,
    to_numpy,
    get_corner,
    lm_cross_entropy_loss,
)  # Helper functions
from easy_transformer.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from easy_transformer import EasyTransformer, EasyTransformerConfig
import easy_transformer
from easy_transformer.experiments import (
    ExperimentMetric,
    AblationConfig,
    EasyAblation,
    EasyPatching,
    PatchingConfig,
)
from transformers import AutoTokenizer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name = "gpt2"  # @param ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'facebook/opt-13b', 'facebook/opt-30b', 'facebook/opt-66b', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B']
model = EasyTransformer.from_pretrained(model_name)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.8/dist-packages/easy_transformer/components.py:616: UserWarning:

Moved LN1 to the attention block



Moving model to device:  cuda
Finished loading pretrained model gpt2 into EasyTransformer!
Moving model to device:  cuda


In [ ]:
from tqdm.auto import trange

### MY CODE STARTS HERE
def get_argmax_for_neuron(modules, embedding_weights, word, start_input, max_passes):
  """
  Gets embedded tensor for which given neuron in given layer has a maximum
  activation value. 
  Args:
    modules - list of transformer modules
    layer - layer for target neuron
    neuron - target neuron index
    start_input - input (embedded tenosr) from which optimisation is started
    max_passes - maximum number of optimisation passes
    threshold - target neuron activation value
  """
#   transformer_blocks = modules
  transformer_blocks = torch.nn.Sequential(*modules).requires_grad_(False)  # *modules[:layer + 1]).requires_grad_(False)
  embedded_tokens = torch.nn.Parameter(start_input.to(device))
  embedded_tokens.requires_grad = True
  optim = torch.optim.SGD([embedded_tokens], lr=0.5e-1)
  token = tokenizer.encode(word, return_tensors="pt").flatten()[0].item()
  bar = trange(max_passes)
  for _ in bar:
    # soft_tokens = embedded_tokens @ embedding_weights.T
    soft_tokens = torch.nn.functional.softmax(embedded_tokens @ embedding_weights.T, dim=-1)
    # quantized_tokens = torch.nn.functional.one_hot((embedded_tokens @ embedding_weights.T).argmax(dim=-1).flatten(), embedding_weights.shape[0]).reshape(soft_tokens.shape).float()
    # quantized_tokens = torch.nn.functional.one_hot(torch.nn.functional.softmax(embedded_tokens @ embedding_weights.T, -1).argmax(dim=-1).flatten(), embedding_weights.shape[0]).reshape(soft_tokens.shape).float()
    # quantized_tokens = torch.nn.functional.one_hot(torch.multinomial(torch.nn.functional.softmax(embedded_tokens @ embedding_weights.T, -1)[0], 1).flatten(), embedding_weights.shape[0]).reshape(soft_tokens.shape).float()

    # z_tokens = quantized_tokens + (soft_tokens - soft_tokens.detach())  # dequantization trick
    z_tokens = soft_tokens
    z_tok = z_tokens @ embedding_weights
    output = transformer_blocks(z_tok) # z_tokens)
    output = torch.nn.functional.log_softmax(output, -1)
    # print(output.exp().sum(-1))
    # Indices of output corresponding to batch, token, neuron. 
    # I decided to take the mean of the outputs over all the tokens,
    # however I am not sure how to interpret that
    # neuron_output = torch.mean(output[0,:,neuron])
    # neuron_output = torch.mean(output[0, :, token])
    neuron_output = torch.mean(output[0, -1, token])
    #print(neuron_output)
    unembedded_tokens, min_dist = get_closest_unembeded_tokens_faster(model, embedded_tokens)
    loss = -neuron_output + embedded_tokens.mean() * 0.1
    # loss = loss - (output.reshape(-1, output.shape[-1])[:-1] * z_tokens[:, 1:].detach()).sum()
    loss.backward()
    optim.step()
    optim.zero_grad()
    # if neuron_output > threshold:
    #   break
    bar.set_postfix(a=-neuron_output.item(), b=output.mean().item(),
                    c="".join(model.to_str_tokens(torch.tensor(unembedded_tokens))))
  return embedded_tokens


def get_closest_unembeded_tokens(model, dream_embeddings):
  """
  Compares given embedded tokens to all tokens from dictionary (50257 tokens),
  and returns a list with closest dictionary tokens corresponding to the 
  embedded tokens. 
  Args:
    model - EasyTransformers GPT2 model instance
    dream_embeddings - tensor of size (1, n_tokens, n_features) for which
    closely positioned tokens are to be found.
  Returns:
    closest_tokens - list of integers of size n_tokens with the tokens
     that are the closest to embedded_dream_tokens
    min_dst - list of floats of size n_tokens with the distances between
     dream tokens and real tokens
  """
  embedding_weights = list(list(model.children())[0].parameters())[0]
  pos_embedding_values= list(list(model.children())[2].parameters())[0]
  
  min_dist = [float('inf') for _ in range(dream_embeddings.size()[1])]
  closest_tokens = [None for _ in range(dream_embeddings.size()[1])]
  for i in range(50257): # Looping over dictionary
    for j, token in enumerate(embedded_tokens[0]): # Looping over positions
      target_tokens = embedding_weights[i] + pos_embedding_values[j]
      distance = torch.sqrt(torch.sum((target_tokens - dream_embeddings[0][j])**2))
      # dont forget to change 'float('inf')' to '-float('inf')' and '<' to '>'
      # if using line below:
      # distance = torch.dot(target_tokens, dream_embeddings[0][j]) 
      if float(distance) < min_dist[j]:
        closest_tokens[j] = i
        min_dist[j] = distance
  return closest_tokens, min_dist


def get_closest_unembeded_tokens_faster(model, dream_embeddings):
  '''
  Compares given embedded tokens to all tokens from dictionary (50257 tokens),
  and returns a list with closest dictionary tokens corresponding to the 
  embedded tokens. Here we ignore the positional embedding and do a 
  dot product with embedding weights
  Args:
    model - EasyTransformers GPT2 model instance
    dream_embeddings - tensor of size (1, n_tokens, n_features) for which
    closely positioned tokens are to be found.
  Returns:
    closest_tokens - list of integers of size n_tokens with the tokens
     that are the closest to embedded_dream_tokens
    min_dst - list of floats of size n_tokens with the distances between
     dream tokens and real tokens
  '''
  embedding_weights = list(list(model.children())[0].parameters())[0]
  pos_embedding_values= list(list(model.children())[2].parameters())[0]
  
  min_dist = [float('inf') for _ in range(dream_embeddings.size()[1])]
  closest_tokens = [None for _ in range(dream_embeddings.size()[1])]

  result = []

  for i, dream_emb in enumerate(dream_embeddings[0]): # looping over positions
    similarities = embedding_weights @ dream_emb
    closest_token = similarities.argmax() # integer
    result.append(closest_token)
  return result, 0

transformer_blocks = [mod for i in [2, 4, 6] for mod in (list(model.children())[i] if hasattr(list(model.children())[i], "__iter__") else [list(model.children())[i]])]
input_shape = (1, 6, 768) # 1 batch, 20 tokens, 768 embedding features
for _ in range(20):
    embedded_tokens = get_argmax_for_neuron(
                                            transformer_blocks,
                                            # model,
                                            list(list(model.children())[0].parameters())[0],
                                            # 0, 0,
                                            "girl",
                                            torch.randn(size=input_shape) * 0.01, 100)
unembedded_tokens, min_dist = get_closest_unembeded_tokens_faster(model, embedded_tokens)
# print("Dreamed tokens: ")
# print("".join(model.to_str_tokens(torch.tensor(unembedded_tokens))))
# print(min_dist)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]